# Part I. ETL Pipeline for Pre-Processing the Files

#### Import Python packages 

In [1]:
# Import Python packages 
import pandas as pd
import cassandra
import re
import os
import glob
import numpy as np
import json
import csv
from prettytable import PrettyTable

#### Creating list of filepaths to process original event csv data files

In [2]:
# checking the current working directory
print(os.getcwd())

# Get the current folder and subfolder event data
filepath = os.getcwd() + '/event_data'

# Create a for loop to create a list of files and collect each filepath
for root, dirs, files in os.walk(filepath):
    
    # join the file path and roots with the subdirectories using glob
    file_path_list = glob.glob(os.path.join(root,'*'))

/home/workspace


#### Processing the files to create the data file csv that will be used for Apache Casssandra tables

In [3]:
# initiating an empty list of rows that will be generated from each file
full_data_rows_list = [] 
    
# for every filepath in the file path list 
for f in file_path_list:

# reading csv file 
    with open(f, 'r', encoding = 'utf8', newline='') as csvfile: 
        # creating a csv reader object 
        csvreader = csv.reader(csvfile) 
        next(csvreader)
        
 # extracting each data row one by one and append it        
        for line in csvreader:
            #print(line)
            full_data_rows_list.append(line) 
            
# creating a smaller event data csv file called event_datafile_full csv that will be used to insert data into the \
# Apache Cassandra tables
csv.register_dialect('myDialect', quoting=csv.QUOTE_ALL, skipinitialspace=True)

with open('event_datafile_new.csv', 'w', encoding = 'utf8', newline='') as f:
    writer = csv.writer(f, dialect='myDialect')
    writer.writerow(['artist','firstName','gender','itemInSession','lastName','length',\
                'level','location','sessionId','song','userId'])
    for row in full_data_rows_list:
        if (row[0] == ''):
            continue
        writer.writerow((row[0], row[2], row[3], row[4], row[5], row[6], row[7], row[8], row[12], row[13], row[16]))


In [4]:
# check the number of rows in the csv file
with open('event_datafile_new.csv', 'r', encoding = 'utf8') as f:
    print(sum(1 for line in f))

6821


# Part II. Creatin Apache Cassandra tables

## We work with <font color=red>event_datafile_new.csv</font>, located within the Workspace directory.  The event_datafile_new.csv contains the following columns: 
- artist 
- firstName of user
- gender of user
- item number in session
- last name of user
- length of the song
- level (paid or free song)
- location of the user
- sessionId
- song title
- userId



In [5]:
file = 'event_datafile_new.csv'

#### Creating a Cluster

In [6]:
# This should make a connection to a Cassandra instance your local machine 
# (127.0.0.1)

from cassandra.cluster import Cluster
cluster = Cluster()

# To establish connection and begin executing queries, need a session
session = cluster.connect()

#### Create Keyspace

In [7]:
try:
    session.execute("""
    CREATE KEYSPACE IF NOT EXISTS udacity 
    WITH REPLICATION = 
    { 'class' : 'SimpleStrategy', 'replication_factor' : 1 }"""
)

except Exception as e:
    print(e)

#### Set Keyspace

In [8]:
try:
    session.set_keyspace('udacity')
except Exception as e:
    print(e)

## Create queries to ask three specific questions to the data

### Query 1: this query must give the artist, song title and song's length in the music app history that was heard during  sessionId = 338, and itemInSession  = 4





In [9]:
# Query 1 definition
query1 = "select artist, song, length from SongsList_session where itemInSession='4' and sessionID='338'"

# Based on query 1, we create 'SongsList_session' table
creation_1 = "CREATE TABLE IF NOT EXISTS SongsList_session"
creation_1 = creation_1 + "(ItemInSession text, sessionID text, artist text, song text, length text, PRIMARY KEY ((itemInSession,sessionID)))" 
try:
    session.execute(creation_1)
except Exception as e:
    print(e)

In [10]:
# We insert recrods inside 'SongsList_session' table, coming from''event_datafile_new.csv'
with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
        query = "INSERT INTO SongsList_session (ItemInSession, sessionID, artist, song, length)"
        query = query + " VALUES (%s, %s, %s, %s, %s)"
        session.execute(query, (line[3], line[8], line[0],line[9],line[5]))

In [11]:
# We display the results of query 1, using PrettyTable
t = PrettyTable(['Artist', 'Song', 'Length'])

try:
    rows = session.execute(query1)
except Exception as e:
    print(e)
    
for row in rows:
    t.add_row([row.artist, row.song, row.length])
    print(t)

+-----------+---------------------------------+----------+
|   Artist  |               Song              |  Length  |
+-----------+---------------------------------+----------+
| Faithless | Music Matters (Mark Knight Dub) | 495.3073 |
+-----------+---------------------------------+----------+


### Query 2: this query must give only the following: name of artist, song (sorted by itemInSession) and user (first and last name) for userid = 10, sessionid = 182
    

In [12]:
# Query 2 definition
query2 = "select artist, song, firstName, lastName from song_playlist_session where userId = '10' and sessionId='182'"

# Based on query 2, we create 'song_playlist_session' table
creation_2 = "CREATE TABLE IF NOT EXISTS song_playlist_session"
creation_2 = creation_2 + "(userId text, sessionId text, ItemInSession text, artist text, song text, firstName text, lastName text, PRIMARY KEY ((userId,sessionId),itemInSession))" 
try:
    session.execute(creation_2)
except Exception as e:
    print(e)                    

In [13]:
# We insert recrods inside 'song_playlist_session' table, coming from''event_datafile_new.csv'
with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
        ## TO-DO: Assign the INSERT statements into the `query` variable
        query = "INSERT INTO song_playlist_session (userId, sessionId, ItemInSession, artist, song, firstName, lastName)"
        query = query + " VALUES (%s, %s, %s, %s, %s, %s, %s)"
        session.execute(query, (line[10], line[8], line[3],line[0],line[9],line[1],line[4]))

In [14]:
# We display the results of query 2, using PrettyTable
t = PrettyTable(['Artist', 'Song', 'First Name', 'Last Name'])

try:
    rows = session.execute(query2)
except Exception as e:
    print(e)
    
for row in rows:
    t.add_row([row.artist, row.song, row.firstname, row.lastname]) 
print(t)

+-------------------+------------------------------------------------------+------------+-----------+
|       Artist      |                         Song                         | First Name | Last Name |
+-------------------+------------------------------------------------------+------------+-----------+
|  Down To The Bone |                  Keep On Keepin' On                  |   Sylvie   |    Cruz   |
|    Three Drives   |                     Greece 2000                      |   Sylvie   |    Cruz   |
| Sebastien Tellier |                      Kilometer                       |   Sylvie   |    Cruz   |
|   Lonnie Gordon   | Catch You Baby (Steve Pitron & Max Sanna Radio Edit) |   Sylvie   |    Cruz   |
+-------------------+------------------------------------------------------+------------+-----------+


### Query 3: this query must give every user name (first and last) in my music app history who listened to the song 'All Hands Against His Own'

In [15]:
# Query 3 definition
query3 = "select firstName, lastName from song_playlist_User where song = 'All Hands Against His Own'"

# Based on query 3, we create 'song_playlist_User' table
creation_3 = "CREATE TABLE IF NOT EXISTS song_playlist_User"
creation_3 = creation_3 + "(song text, userId text, firstName text, lastName text, PRIMARY KEY (song, userId))" 
try:
    session.execute(creation_3)
except Exception as e:
    print(e)                             

In [16]:
# We insert recrods inside 'song_playlist_User' table, coming from''event_datafile_new.csv'
with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
        query = "INSERT INTO song_playlist_User (song, userId, firstName, lastName)"
        query = query + " VALUES (%s, %s, %s, %s)"
        session.execute(query, (line[9], line[10],line[1], line[4]))  

In [17]:
# We display the results of query 3, using PrettyTable
t = PrettyTable(['First Name', 'Last Name'])

try:
    rows = session.execute(query3)
except Exception as e:
    print(e)

for row in rows:
    t.add_row([row.firstname, row.lastname])     
print(t)

+------------+-----------+
| First Name | Last Name |
+------------+-----------+
| Jacqueline |   Lynch   |
|   Tegan    |   Levine  |
|    Sara    |  Johnson  |
+------------+-----------+


### Drop the tables before closing out the sessions

In [18]:
query = "drop table SongsList_session"
try:
    rows = session.execute(query)
except Exception as e:
    print(e)
    
query = "drop table song_playlist_session"
try:
    rows = session.execute(query)
except Exception as e:
    print(e)
    
query = "drop table song_playlist_User"
try:
    rows = session.execute(query)
except Exception as e:
    print(e)

### Close the session and cluster connection¶

In [19]:
session.shutdown()
cluster.shutdown()